# Constrained Optimization

Find the minimum of the following quadratic function on $\mathbb{R}^2$ 

$$f(x) = x^TAx +b^Tx +c$$
where
$$A = \left(\begin{matrix}13&5\\5&7\end{matrix}\right), b = \left(\begin{matrix}1\\1\end{matrix}\right) \textrm {and } c = 2$$

Under the constraints:
$$g(x) = 2x_1-5x_2=2 \;\;\;\;\;\; \textrm{ and } \;\;\;\;\;\; h(x) = x_1+x_2=1$$

1. Use a matrix decomposition method to find the minimum of the *unconstrained* problem without using `scipy.optimize` (Use library functions - no need to code your own). Note: for full credit you should exploit matrix structure. 
2. Find the solution using constrained optimization with the `scipy.optimize` package. 
2. Use Lagrange multipliers and solving the resulting set of equations directly without using `scipy.optimize`. 

## Solve unconstrained problem

To find the minimum, we differentiate $f(x)$ with respect to $x^T$ and set it equal to $0$. We thus need to solve

$$
2Ax + b = 0
$$

or

$$ 
Ax = -\frac{b}{2}
$$

We see that $A$ is a symmetric positive definite real matrix. Hence we use Cholesky factorization.

In [1]:
import numpy as np

In [2]:
from scipy.linalg import cholesky, solve_triangular, cho_solve, cho_factor

In [3]:
A = np.array([
    [13,5],
    [5,7]
])
b = np.array([1,1]).reshape(-1,1)
c = 2

### Solve using Cholesky 

Steps

$$
L = \text{cholesky}(A) \\
\text{solve } Ly = b \\
\text{solve } L^Tx = y
$$

In [4]:
L = cholesky(A, lower=True)
y = solve_triangular(L, -b/2, lower=True)
x = solve_triangular(L.T, y, lower=False)

In [5]:
x

array([[-0.01515152],
       [-0.06060606]])

#### Short cut

In [6]:
cho_solve(cho_factor(A), -b/2)

array([[-0.01515152],
       [-0.06060606]])

## Solve constrained problem using `scipy.optimize`

In [7]:
f = lambda x, A, b, c: x.T @ A @ x + b.T @ x + c

In [8]:
from scipy.optimize import minimize

In [9]:
cons = ({'type': 'eq', 'fun': lambda x: 2*x[0] - 5*x[1] - 2},
        {'type': 'eq', 'fun': lambda x: x[0] + x[1] - 1})

res = minimize(f, [0,0], constraints=cons, args=(A, b, c))
res.x

array([1.00000000e+00, 3.41607086e-16])

## Solve constrained problem using Lagrange multipliers

We want to solve the following problem

$$
L(x_1, x_2, \lambda, \mu) = \nabla f + \lambda \nabla g + \mu \nabla h = 0 \\
g(x) = 2 \\
h(x) = 1
$$

Sometimes the Lagrangian is written as 

$$
L(x_1, x_2, \lambda, \mu) = \nabla f - \lambda \nabla g - \mu \nabla h
$$

All this means is a final sign change in the estimated values of $\lambda$ and $\mu$.

This can be written in matrix from as 

$$
\begin{bmatrix}
\frac{\delta f}{\delta x_1} & \frac{\delta f}{\delta x_2} & \frac{\delta f}{\delta \lambda} & \frac{\delta f}{\delta \mu} \\
2 & -5 & 0 & 0 \\
1 & 1 & 0 & 0 
\end{bmatrix} \begin{bmatrix}
x_1 \\
x_2 \\
\lambda \\
\mu
\end{bmatrix} = \begin{bmatrix}
-1 \\
-1 \\
2 \\
1
\end{bmatrix} 
$$

Plugging in the numbers, we finally get

$$
\begin{bmatrix}
26 & 10 & 2 & 1 \\
10 & 14 & -5 & 1 \\
2 & -5 & 0 & 0 \\
1 & 1 & 0 & 0 
\end{bmatrix} \begin{bmatrix}
x_1 \\
x_2 \\
\lambda \\
\mu
\end{bmatrix} = \begin{bmatrix}
-1 \\
-1 \\
2 \\
1
\end{bmatrix}
$$

In [10]:
from scipy.linalg import solve

In [11]:
A = np.array([
    [26, 10, 2, 1],
    [10, 14, -5, 1],
    [2, -5, 0, 0],
    [1, 1, 0, 0]
])
b = np.array([-1, -1, 2, 1]).reshape(-1,1)

In [12]:
solve(A, b)

array([[ 1.00000000e+00],
       [-4.37360585e-17],
       [-2.28571429e+00],
       [-2.24285714e+01]])